# Full end-to-end integration of our solution.  Each test image is first put through the trained CNN model to determine what the machine type is.  The image is then put through the OpenCV EAST model to find the potential location of all text in the image.  We then use the pixel data to create a new dataset using the relartive pixel locations to each other.  This data is then combined with the predicted machine type put put through the trained MLP.  Each and every identified text location is predcited as either PII or nonPII.  If predicted as PII, the program then blacks out that data prior to saving the edited photo.  nothing is done if the data is predicted to be nonPII.

In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
#import cv2
import pandas as pd
import pickle
import random
import time
import psutil
from scipy.spatial import distance

# Import required modules
import cv2 as cv
import math
import argparse


confThreshold = 0.00000001
nmsThreshold = 0.5
inpWidth = 320
inpHeight = 320
model = 'frozen_east_text_detection.pb'

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
import glob
df = pickle.load( open( "/home/ubuntu/efs/acrin_by_machine/machine_photos/df_save_full.pkl", "rb" ) )
#df['machine']=0
for index,row in df.iterrows():
    df.loc[index,'machine'] = row.fileName.split('/')[-2][-1]

In [4]:
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.vgg19 import VGG19
import tensorflow as tf

CLASSES = 6
    
# setup model
base_model = InceptionV3(weights='imagenet', include_top=False)
#base_model = VGG19(weights='imagenet', include_top=False)

# Add more layers
machine_model = Sequential()
machine_model.add(base_model)
machine_model.add(GlobalAveragePooling2D(name='avg_pool'))

machine_model.add(Dense(CLASSES, activation='softmax'))
#model = Model(inputs=base_model.input, outputs=predictions)
   
# transfer learning - set all layers of the base model to frozen
for layer in base_model.layers:
    layer.trainable = False

# Unfreeze the last x layers from bottom
x = 11
for layer in base_model.layers[-x:]:
     layer.trainable = True
    
machine_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

machine_model.load_weights('/home/ubuntu/efs/acrin_by_machine/machine_photos/saved_weights_0_hdf5')


Using TensorFlow backend.
W1203 16:51:18.064619 140394239682368 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1203 16:51:18.093061 140394239682368 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1203 16:51:18.099294 140394239682368 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1203 16:51:18.123700 140394239682368 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensor

In [6]:
def create_test_data(num):
    WIDTH = 299
    HEIGHT = 299
    BATCH_SIZE = 128

    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.gridspec as gridspec
    import os
    from PIL import Image
    from keras.preprocessing import image
    from keras.models import load_model
    from tqdm import tqdm
    import cv2

    X = []
    X_p = []
    Y = []
    files = []
    image_name = []

    img = image.load_img(df.iloc[num].fileName, target_size=(HEIGHT, WIDTH))
    x = image.img_to_array(img)
    x = cv2.flip(x, 1 )
    x = cv2.flip(x, 0 )
    x = preprocess_input(x)
    X.append(np.array(x))
    X=np.array(X)
    return(X)

In [7]:
test_list = df.index[(df['use'] == 'test') & (df['good'] == 1)].tolist()

In [8]:
X = []

for i in test_list:
    temp = create_train_array(i)
    temp = temp.reshape((299,299,3))
    X.append(temp)
    print(int(df.iloc[i].machine)-1,end=' ')
X=np.array(X)
machine_pred = machine_model.predict(X)
machine_pred = np.argmax(machine_pred, axis=1)
machine_pred

NameError: name 'create_train_array' is not defined

In [9]:
df.iloc[100]

fileName    /home/ubuntu/efs/acrin_by_machine/machine_phot...
good                                                        1
machc                                                       0
use                                                     valid
path        /home/ubuntu/efs/acrin_by_machine/machine_phot...
machine                                                     1
Name: 100, dtype: object

In [10]:
temp = temp.reshape((299,299,3))
temp.shape

NameError: name 'temp' is not defined

In [66]:
def is_in_rect_mach1(bc,ec,br,er,centr,centc,old_size,ratio,sc,sr,new_im):

    new_size = tuple([int(x*ratio) for x in old_size])

    top = int((old_size[0] - new_size[0])/2)
    bottom = int((old_size[0] - new_size[0])/2)
    left = int((old_size[1] - new_size[1])/2)
    right = int((old_size[1] - new_size[1])/2)

    box1_ulc = left+int(650*ratio)+sc
    box1_ulr = top+int(550*ratio)+sr

    box1_lrc = right + int(1450*ratio)+sc
    box1_lrr = bottom + int(650*ratio)+sr
    
    
    box2_ulc = left+int(650*ratio)+sc
    box2_ulr = top+int(650*ratio)+sr

    box2_lrc = right + int(1450*ratio)+sc
    box2_lrr = bottom + int(750*ratio)+sr


    box3_ulc = left+int(2750*ratio)+sc
    box3_ulr = top+int(620*ratio)+sr

    box3_lrc = right + int(3400*ratio)+sc
    box3_lrr = bottom + int(705*ratio)+sr

    #print(ratio,sc,sr)
    
    if (box1_ulc < centc < box1_lrc) & (box1_ulr < centr < box1_lrr):
        return(True)
    if (box2_ulc < centc < box2_lrc) & (box2_ulr < centr < box2_lrr):
        return(True)
    if (box3_ulc < centc < box3_lrc) & (box3_ulr < centr < box3_lrr):
        return(True)


In [67]:
def is_in_rect_mach2(bc,ec,br,er,centr,centc,old_size,ratio,sc,sr,new_im):

    new_size = tuple([int(x*ratio) for x in old_size])

    top = int((old_size[0] - new_size[0])/2)
    bottom = int((old_size[0] - new_size[0])/2)
    left = int((old_size[1] - new_size[1])/2)
    right = int((old_size[1] - new_size[1])/2)


    box1_ulc = left+int(530*ratio)+sc
    box1_ulr = top+int(550*ratio)+sr

    box1_lrc = right + int(1485*ratio)+sc
    box1_lrr = bottom + int(655*ratio)+sr
    
    
    box2_ulc = left+int(530*ratio)+sc
    box2_ulr = top+int(650*ratio)+sr

    box2_lrc = right + int(1485*ratio)+sc
    box2_lrr = bottom + int(730*ratio)+sr


    box3_ulc = left+int(1495*ratio)+sc
    box3_ulr = top+int(550*ratio)+sr

    box3_lrc = right + int(2620*ratio)+sc
    box3_lrr = bottom + int(655*ratio)+sr

    
    box4_ulc = left+int(1495*ratio)+sc
    box4_ulr = top+int(650*ratio)+sr


    box4_lrc = right + int(2620*ratio)+sc
    box4_lrr = bottom + int(730*ratio)+sr



    
    if (box1_ulc < centc < box1_lrc) & (box1_ulr < centr < box1_lrr):
        return(True)
    if (box2_ulc < centc < box2_lrc) & (box2_ulr < centr < box2_lrr):
        return(True)
    if (box3_ulc < centc < box3_lrc) & (box3_ulr < centr < box3_lrr):
        return(True)
    if (box4_ulc < centc < box4_lrc) & (box4_ulr < centr < box4_lrr):
        return(True)

    

In [68]:
def is_in_rect_mach3(bc,ec,br,er,centr,centc,old_size,ratio,sc,sr,new_im):

    new_size = tuple([int(x*ratio) for x in old_size])

    top = int((old_size[0] - new_size[0])/2)
    bottom = int((old_size[0] - new_size[0])/2)
    left = int((old_size[1] - new_size[1])/2)
    right = int((old_size[1] - new_size[1])/2)

    box1_ulc = left+int(351*ratio)+sc
    box1_ulr = top+int(470*ratio)+sr

    box1_lrc = right + int(1450*ratio)+sc
    box1_lrr = bottom + int(549*ratio)+sr
    
    
    box2_ulc = left+int(1455*ratio)+sc
    box2_ulr = top+int(465*ratio)+sr

    box2_lrc = right + int(2440*ratio)+sc
    box2_lrr = bottom + int(544*ratio)+sr


    box3_ulc = left+int(351*ratio)+sc
    box3_ulr = top+int(560*ratio)+sr

    box3_lrc = right + int(1450*ratio)+sc
    box3_lrr = bottom + int(638*ratio)+sr

    
    box4_ulc = left+int(2445*ratio)+sc
    box4_ulr = top+int(460*ratio)+sr


    box4_lrc = right + int(2900*ratio)+sc
    box4_lrr = bottom + int(539*ratio)+sr


    box5_ulc = left+int(2445*ratio)+sc
    box5_ulr = top+int(550*ratio)+sr

    box5_lrc = right + int(2900*ratio)+sc
    box5_lrr = bottom + int(628*ratio)+sr


    
    if (box1_ulc < centc < box1_lrc) & (box1_ulr < centr < box1_lrr):
        return(True)
    if (box2_ulc < centc < box2_lrc) & (box2_ulr < centr < box2_lrr):
        return(True)
    if (box3_ulc < centc < box3_lrc) & (box3_ulr < centr < box3_lrr):
        return(True)
    if (box4_ulc < centc < box4_lrc) & (box4_ulr < centr < box4_lrr):
        return(True)
    if (box5_ulc < centc < box5_lrc) & (box5_ulr < centr < box5_lrr):
        return(True)
    

In [69]:
def is_in_rect_mach4(bc,ec,br,er,centr,centc,old_size,ratio,sc,sr,new_im):

    new_size = tuple([int(x*ratio) for x in old_size])

    top = int((old_size[0] - new_size[0])/2)
    bottom = int((old_size[0] - new_size[0])/2)
    left = int((old_size[1] - new_size[1])/2)
    right = int((old_size[1] - new_size[1])/2)


    box1_ulc = left+int(950*ratio)+sc
    box1_ulr = top+int(450*ratio)+sr

    box1_lrc = right + int(2050*ratio)+sc
    box1_lrr = bottom + int(549*ratio)+sr
    

    box2_ulc = left+int(530*ratio)+sc
    box2_ulr = top+int(549*ratio)+sr

    box2_lrc = right + int(1550*ratio)+sc
    box2_lrr = bottom + int(648*ratio)+sr


    box3_ulc = left+int(2050*ratio)+sc
    box3_ulr = top+int(450*ratio)+sr

    box3_lrc = right + int(2890*ratio)+sc
    box3_lrr = bottom + int(544*ratio)+sr

    
    box4_ulc = left+int(1550*ratio)+sc
    box4_ulr = top+int(550*ratio)+sr


    box4_lrc = right + int(2480*ratio)+sc
    box4_lrr = bottom + int(648*ratio)+sr



    
    if (box1_ulc < centc < box1_lrc) & (box1_ulr < centr < box1_lrr):
        return(True)
    if (box2_ulc < centc < box2_lrc) & (box2_ulr < centr < box2_lrr):
        return(True)
    if (box3_ulc < centc < box3_lrc) & (box3_ulr < centr < box3_lrr):
        return(True)
    if (box4_ulc < centc < box4_lrc) & (box4_ulr < centr < box4_lrr):
        return(True)

    

In [70]:
def is_in_rect_mach5(bc,ec,br,er,centr,centc,old_size,ratio,sc,sr,new_im):

    new_size = tuple([int(x*ratio) for x in old_size])

    top = int((old_size[0] - new_size[0])/2)
    bottom = int((old_size[0] - new_size[0])/2)
    left = int((old_size[1] - new_size[1])/2)
    right = int((old_size[1] - new_size[1])/2)
    
    
    box1_ulc = left+int(500*ratio)+sc
    box1_ulr = top+int(650*ratio)+sr

    box1_lrc = right + int(1400*ratio)+sc
    box1_lrr = bottom + int(750*ratio)+sr
    
    
    box2_ulc = left+int(500*ratio)+sc
    box2_ulr = top+int(750*ratio)+sr

    box2_lrc = right + int(1400*ratio)+sc
    box2_lrr = bottom + int(850*ratio)+sr


    box3_ulc = left+int(1400*ratio)+sc
    box3_ulr = top+int(650*ratio)+sr

    box3_lrc = right + int(2300*ratio)+sc
    box3_lrr = bottom + int(750*ratio)+sr

    
    box4_ulc = left+int(2600*ratio)+sc
    box4_ulr = top+int(650*ratio)+sr


    box4_lrc = right + int(3300*ratio)+sc
    box4_lrr = bottom + int(750*ratio)+sr


    box5_ulc = left+int(2600*ratio)+sc
    box5_ulr = top+int(750*ratio)+sr

    box5_lrc = right + int(3300*ratio)+sc
    box5_lrr = bottom + int(850*ratio)+sr


    
    if (box1_ulc < centc < box1_lrc) & (box1_ulr < centr < box1_lrr):
        return(True)
    if (box2_ulc < centc < box2_lrc) & (box2_ulr < centr < box2_lrr):
        return(True)
    if (box3_ulc < centc < box3_lrc) & (box3_ulr < centr < box3_lrr):
        return(True)
    if (box4_ulc < centc < box4_lrc) & (box4_ulr < centr < box4_lrr):
        return(True)
    if (box5_ulc < centc < box5_lrc) & (box5_ulr < centr < box5_lrr):
        return(True)
    

In [71]:
def is_in_rect_mach6(bc,ec,br,er,centr,centc,old_size,ratio,sc,sr,new_im):

    new_size = tuple([int(x*ratio) for x in old_size])

    top = int((old_size[0] - new_size[0])/2)
    bottom = int((old_size[0] - new_size[0])/2)
    left = int((old_size[1] - new_size[1])/2)
    right = int((old_size[1] - new_size[1])/2)
    
    box1_ulc = left+int(300*ratio)+sc
    box1_ulr = top+int(450*ratio)+sr

    box1_lrc = right + int(1100*ratio)+sc
    box1_lrr = bottom + int(520*ratio)+sr
    
    
    box2_ulc = left+int(300*ratio)+sc
    box2_ulr = top+int(520*ratio)+sr

    box2_lrc = right + int(1100*ratio)+sc
    box2_lrr = bottom + int(590*ratio)+sr


    box3_ulc = left+int(300*ratio)+sc
    box3_ulr = top+int(590*ratio)+sr

    box3_lrc = right + int(1100*ratio)+sc
    box3_lrr = bottom + int(660*ratio)+sr

    
    box4_ulc = left+int(300*ratio)+sc
    box4_ulr = top+int(660*ratio)+sr


    box4_lrc = right + int(1100*ratio)+sc
    box4_lrr = bottom + int(730*ratio)+sr

    box5_ulc = left+int(2280*ratio)+sc
    box5_ulr = top+int(430*ratio)+sr


    box5_lrc = right + int(3400*ratio)+sc
    box5_lrr = bottom + int(505*ratio)+sr
    
    if (box1_ulc < centc < box1_lrc) & (box1_ulr < centr < box1_lrr):
        return(True)
    if (box2_ulc < centc < box2_lrc) & (box2_ulr < centr < box2_lrr):
        return(True)
    if (box3_ulc < centc < box3_lrc) & (box3_ulr < centr < box3_lrr):
        return(True)
    if (box4_ulc < centc < box4_lrc) & (box4_ulr < centr < box4_lrr):
        return(True)
    if (box5_ulc < centc < box5_lrc) & (box5_ulr < centr < box5_lrr):
        return(True)

In [11]:
def decode(scores, geometry, scoreThresh,img_name):
    detections = []
    confidences = []
    
    
    ############ CHECK DIMENSIONS AND SHAPES OF geometry AND scores ############
    assert len(scores.shape) == 4, "Incorrect dimensions of scores"
    assert len(geometry.shape) == 4, "Incorrect dimensions of geometry"
    assert scores.shape[0] == 1, "Invalid dimensions of scores"
    assert geometry.shape[0] == 1, "Invalid dimensions of geometry"
    assert scores.shape[1] == 1, "Invalid dimensions of scores"
    assert geometry.shape[1] == 5, "Invalid dimensions of geometry"
    assert scores.shape[2] == geometry.shape[2], "Invalid dimensions of scores and geometry"
    assert scores.shape[3] == geometry.shape[3], "Invalid dimensions of scores and geometry"
    height = scores.shape[2]
    width = scores.shape[3]
    for y in range(0, height):

        # Extract data from scores
        scoresData = scores[0][0][y]
        x0_data = geometry[0][0][y]
        x1_data = geometry[0][1][y]
        x2_data = geometry[0][2][y]
        x3_data = geometry[0][3][y]
        anglesData = geometry[0][4][y]
        for x in range(0, width):
            score = scoresData[x]

            # If score is lower than threshold score, move to next x
            if(score < scoreThresh):
                continue

            # Calculate offset
            offsetX = x * 4.0
            offsetY = y * 4.0
            angle = anglesData[x]

            # Calculate cos and sin of angle
            cosA = math.cos(angle)
            sinA = math.sin(angle)
            h = x0_data[x] + x2_data[x]
            w = x1_data[x] + x3_data[x]

            # Calculate offset
            offset = ([offsetX + cosA * x1_data[x] + sinA * x2_data[x], offsetY - sinA * x1_data[x] + cosA * x2_data[x]])

            # Find points for rectangle
            p1 = (-sinA * h + offset[0], -cosA * h + offset[1])
            p3 = (-cosA * w + offset[0],  sinA * w + offset[1])
            center = (0.5*(p1[0]+p3[0]), 0.5*(p1[1]+p3[1]))
            #center = (0.5*(p1[0]+p3[0])+w_off, 0.5*(p1[1]+p3[1])+h_off)
            #detections.append((center, (w,h), -1*angle * 180.0 / math.pi))
            detections.append((center, (w,h),0.0))
            
            confidences.append(float(score))

    # Return detections and confidences
    return [detections, confidences]

In [12]:
def create_data_points(this_image,test,boxx):
    confThreshold = 0.00005
    nmsThreshold = 0.25
    inpWidth = 320
    inpHeight = 320
    import cv2
    slices = 3

    net = cv.dnn.readNet('frozen_east_text_detection.pb')
    outputLayers = []
    outputLayers.append("feature_fusion/Conv_7/Sigmoid")
    outputLayers.append("feature_fusion/concat_3")

    point_data = []
    inpWidth = 320
    inpHeight = 320    
    # Open a video file or an image file or a camera stream
    cap = cv.VideoCapture(this_image)
    hasFrame, frame = cap.read()
    # Get frame height and width
    height_ = frame.shape[0]
    width_ = frame.shape[1]
    rW = width_ / float(inpWidth)
    rH = height_ / float(inpHeight)
    # Create a 4D blob from frame.
    blob = cv.dnn.blobFromImage(frame, 1.0, (inpWidth, inpHeight), (123.68, 116.78, 103.94), True, False)

    # Run the model
    net.setInput(blob)
    output = net.forward(outputLayers)
    t, _ = net.getPerfProfile()
    label = 'Inference time: %.2f ms' % (t * 1000.0 / cv.getTickFrequency())
    # Get scores and geometry
    scores = output[0]
    geometry = output[1]
    [boxes, confidences] = decode(scores, geometry, confThreshold,this_image)
    # Apply NMS
    indices = cv.dnn.NMSBoxesRotated(boxes, confidences, confThreshold,nmsThreshold)
    #print(confThreshold)
    #print(nmsThreshold)
    row_off = (this_image.split('/')[-1].split('_')[-2])
    col_off = (this_image.split('/')[-1].split('_')[-3])
    #print(row_off,col_off) 
    for i in indices:
        # get 4 corners of the rotated rect
        vertices = cv.boxPoints(boxes[i[0]])
        # scale the bounding box coordinates based on the respective ratios
        c1 = 0
        c2 = 0
        r1 = 0
        r2 = 0
        for j in range(4):
            vertices[j][0] *= rW
            vertices[j][1] *= rH
        for j in range(4):
            p1 = (int(vertices[j][0]+int(row_off)), int(vertices[j][1]+int(col_off)))
            p2 = (int(vertices[(j + 1) % 4][0]+int(row_off)), int(vertices[(j + 1) % 4][1]+int(col_off)))
            #print(p1,p2)
            if j == 0:
                c1 = p1[0]
                r1 = p1[1]
            if  c1 != p2[0]:
                c2 = p2[0]
            if r1 != p2[1]:
                r2 = p2[1]
        if c1 < c2:
            bc = c1
            ec = c2
        else:
            bc = c2
            ec = c1
        if r1 < r2:
            br = r1
            er = r2
        else:
            br = r2
            er = r1
        #print(bc,br,ec,er)
        centr = br + int((er-br)/2)
        centc = bc + int((ec-bc)/2)
        #print(centc,centr)
        if (centr>0) & (centc>0):
            test.append((centc,centr))
            boxx.append([bc,br,ec,er])
            #cv.rectangle(img, (bc,br),(ec,er), (0,255,0), 10)

    return(test,boxx)

In [31]:
def create_df(num):
    confThreshold = 0.00005
    nmsThreshold = 0.25
    inpWidth = 320
    inpHeight = 320
    import cv2
    slices = 3

    net = cv.dnn.readNet('frozen_east_text_detection.pb')
    outputLayers = []
    outputLayers.append("feature_fusion/Conv_7/Sigmoid")
    outputLayers.append("feature_fusion/concat_3")

    count = 0
    scale_ratio=.85
    scale_sr=.1
    scale_sc=.1

    data_df = pd.DataFrame(columns=['fileName','ratio','sr','sc','machine', 'boxx'])
    this_mach = df.iloc[image_num].fileName

    this_frame = df.iloc[image_num]
    my_image=this_frame.fileName
    stime=time.time()
    img = cv.imread(my_image)
    
    plt.figure(figsize=(30,20))
    plt.imshow(img)
    plt.show()
    
    ratio=rand_scale = random.uniform(scale_ratio,1)
    rand_scale = random.uniform(-scale_sr,scale_sr)
    sr=int(img.shape[0]*rand_scale)
    rand_scale = random.uniform(-scale_sc,scale_sc)
    sc=int(img.shape[1]*rand_scale)

    old_size = img.shape[:2] # old_size is in (height, width) format

    new_size = tuple([int(x*ratio) for x in old_size])

    # new_size should be in (width, height) format

    im = cv.resize(img, (new_size[1], new_size[0]))

    top = int((old_size[0] - new_size[0])/2)
    bottom = int((old_size[0] - new_size[0])/2)
    left = int((old_size[1] - new_size[1])/2)
    right = int((old_size[1] - new_size[1])/2)

    if top+bottom+new_size[0] < old_size[0]:
        top += 1
    if left+right+new_size[1] < old_size[1]:
        left += 1

    color = [0, 0, 0]
    img= cv.copyMakeBorder(im, top, bottom, left, right, cv.BORDER_CONSTANT,
        value=color)


    rows,cols,_ = img.shape
    M = np.float32([[1,0,sc],[0,1,sr]])
    img = cv.warpAffine(img,M,(cols,rows))

    split_1 = int(img.shape[0]/slices)
    split_2 = int(img.shape[1]/slices)
    for r in range(0,img.shape[0],split_1):
        for c in range(0,img.shape[1],split_2):
            cv2.imwrite('/home/ubuntu/opencveast/img_'+str(r)+'_'+str(c)+'_.png',img[r:r+split_1, c:c+split_2,:])

            nr = r + int((img.shape[0]/slices)/2)
            nc = c + int((img.shape[1]/slices)/2)
            cv2.imwrite('/home/ubuntu/opencveast/img_'+str(r)+'_'+str(nc)+'_.png',img[r:r+split_1, nc:nc+split_2,:])
            cv2.imwrite('/home/ubuntu/opencveast/img_'+str(nr)+'_'+str(c)+'_.png',img[nr:nr+split_1, c:c+split_2,:])
    img_list = glob.glob('/home/ubuntu/opencveast/img*')
    cnt = 1
    test = []
    boxx = []
    for this_image in img_list:
        test,boxx = create_data_points(this_image,test,boxx)

    data_df = data_df.append({'fileName': this_frame.fileName,
                                'machine':machine_pred[0]+1, 
                                'point_data': test,
                                'boxx' : boxx,
                                'ratio' :ratio,
                                'sr':sr,
                                'sc':sc
                               }, ignore_index=True)
    return(data_df)

In [14]:
def create_data_set(test_df,mach_pred,image_num):    
    test_data_set = pd.DataFrame(columns=['machine','ul','ll','ur','lr','ulx','llx','urx','lrx','uly','lly','ury','lry','ed_ul','ed_ll','ed_ur','ed_lr',
                                     'vd_ul','vd_ll','vd_ur','vd_lr','hd_ul','hd_ll','hd_ur','hd_lr','class'])
    #try:
    img = cv.imread(test_df.loc[0].fileName)
    this_item = test_df.loc[0].point_data
    this_boxx = test_df.loc[0].boxx

    dist_list = []
    for item in this_item:
        dist_list.append(distance.euclidean((0,0),item))
    ul = np.argmin(dist_list)
    dist_list = []
    for item in this_item:
        dist_list.append(distance.euclidean((0,img.shape[0]),item))
    ll = np.argmin(dist_list)
    dist_list = []
    for item in this_item:
        dist_list.append(distance.euclidean((img.shape[1],0),item))
    ur = np.argmin(dist_list)
    dist_list = []
    for item in this_item:
        dist_list.append(distance.euclidean((img.shape[1],img.shape[0]),item))
    lr = np.argmin(dist_list)

    for item in range(len(this_item)):
        test_data_set = test_data_set.append({'machine':test_df.loc[0].machine,
                                    'class':1,
                                    'ul':dist_list[ul],
                                    'll':dist_list[ll],
                                    'ur':dist_list[ur],
                                    'lr':dist_list[lr],
                                    'ulx':this_item[ul][0],
                                    'llx':this_item[ll][0],
                                    'urx':this_item[ur][0],
                                    'lrx':this_item[lr][0],
                                    'uly':this_item[ul][1],
                                    'lly':this_item[ll][1],
                                    'ury':this_item[ur][1],
                                    'lry':this_item[lr][1],
                                    'boxx' : this_boxx[item],
                                    'ed_ul':distance.euclidean(0,this_item[ul],this_item[item]),
                                    'ed_ll':distance.euclidean(0,this_item[ll],this_item[item]),
                                    'ed_ur':distance.euclidean(0,this_item[ur],this_item[item]),
                                    'ed_lr':distance.euclidean(0,this_item[lr],this_item[item]),
                                    'vd_ul':this_item[ul][0]-this_item[item][0],
                                    'vd_ll':this_item[ll][0]-this_item[item][0],
                                    'vd_ur':this_item[ur][0]-this_item[item][0],
                                    'vd_lr':this_item[lr][0]-this_item[item][0],
                                    'hd_ul':this_item[ul][1]-this_item[item][1],
                                    'hd_ll':this_item[ll][1]-this_item[item][1],
                                    'hd_ur':this_item[ur][1]-this_item[item][1],
                                    'hd_lr':this_item[lr][1]-this_item[item][1]
                                       }, ignore_index=True)

    #except:
    #    pass
    return(test_data_set)


In [15]:
import dill
with open('/home/ubuntu/test_df_points.pkl','rb') as fp:
    data_df=dill.load(fp)

In [16]:
data_df.head()

,fileName,ratio,sr,sc,machine,piidata,nopiidata
0,/home/ubuntu/efs/acrin_by_machine/machine_phot...,0.906229,-59,175,1,"[(3055, 693), (3323, 687), (1448, 711), (1448,...","[(3475, 769), (3372, 1434), (3517, 1429), (333..."
1,/home/ubuntu/efs/acrin_by_machine/machine_phot...,0.878613,95,-2,2,"[(1884, 798), (1454, 872), (1746, 876), (2022,...","[(3073, 1244), (3161, 790), (3191, 874), (2936..."
2,/home/ubuntu/efs/acrin_by_machine/machine_phot...,0.853372,-130,42,3,"[(1712, 530), (2600, 524), (2560, 598), (2478,...","[(3323, 599), (3198, 596), (3177, 520), (864, ..."
3,/home/ubuntu/efs/acrin_by_machine/machine_phot...,0.885532,242,120,4,"[(2823, 845), (2392, 845), (1914, 959), (1453,...","[(3105, 840), (2866, 942), (3338, 839), (3381,..."
4,/home/ubuntu/efs/acrin_by_machine/machine_phot...,0.910913,-170,259,5,"[(3266, 616), (3221, 682), (3390, 683), (1649,...","[(3067, 818), (3337, 1503), (3219, 766), (1776..."


In [17]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(8, input_dim=25, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.load_weights('weights_image_final_v2')
import dill
with open('/home/ubuntu/train_mean_mach.pkl','rb') as fp:
    mean=dill.load(fp)
with open('/home/ubuntu/train_std_mach.pkl','rb') as fp:
    std=dill.load(fp)

W1203 16:52:53.568477 140394239682368 deprecation.py:323] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [18]:
import numpy as np
from sklearn.metrics import accuracy_score

predicts = model.predict(X)
predicts = np.argmax(predicts, axis=1)

accuracy_score(Y, predicts)

IndexError: list index out of range

# Code for spot-checking data points

In [33]:
def print_image(i,Te):
    img = cv.imread(data_df.loc[0].fileName)
    ratio=rand_scale = data_df.loc[0].ratio
    sr=data_df.loc[0].sr
    sc=data_df.loc[0].sc

    old_size = img.shape[:2] # old_size is in (height, width) format

    new_size = tuple([int(x*ratio) for x in old_size])

    # new_size should be in (width, height) format

    im = cv.resize(img, (new_size[1], new_size[0]))

    top = int((old_size[0] - new_size[0])/2)
    bottom = int((old_size[0] - new_size[0])/2)
    left = int((old_size[1] - new_size[1])/2)
    right = int((old_size[1] - new_size[1])/2)

    if top+bottom+new_size[0] < old_size[0]:
        top += 1
    if left+right+new_size[1] < old_size[1]:
        left += 1

    color = [0, 0, 0]
    img= cv.copyMakeBorder(im, top, bottom, left, right, cv.BORDER_CONSTANT,
        value=color)

    rows,cols,_ = img.shape
    M = np.float32([[1,0,sc],[0,1,sr]])
    img = cv.warpAffine(img,M,(cols,rows))
    
    plt.figure(figsize=(30,20))
    plt.imshow(img)
    plt.show()
    
    probs = model.predict(Te)

    for this_point in range(len(test_data_set)):
        box = test_data_set.iloc[this_point].boxx
        if probs[this_point][0] > .5:
            cv.rectangle(img, (box[0],box[1]),(box[2],box[3]), (0,0,0), -10)

    plt.figure(figsize=(30,20))
    plt.imshow(img)
    plt.show()




In [17]:
image_num = 200
data_df= create_df(image_num)

NameError: name 'create_data_points' is not defined

In [1]:
for image_num in range(10):
    
    image_num = random.randint(0,len(df))
    X = create_test_data(image_num)
    machine_pred = machine_model.predict(X)
    machine_pred = np.argmax(machine_pred, axis=1)
    data_df= create_df(image_num)
    test_data_set = create_data_set(data_df,machine_pred,image_num)

    test_data = test_data_set.to_numpy()
    Te = test_data[:,:-2]
    y_t = test_data[:,-2]
    Te = Te.astype(float)
    y_t = y_t.astype(float)
    Te = [(x - mean)/std for x in Te]
    Te = np.array(Te)

    print_image(image_num,Te)

NameError: name 'random' is not defined

In [1]:
test_data_set.head()

NameError: name 'test_data_set' is not defined

In [27]:
import numpy as np
from sklearn.metrics import accuracy_score
new_y = []
new_predicts = []
predicts = model.predict(Te)
for i in range(len(predicts)):
    new_predicts.append(round(predicts[i][0]))
    #new_y.append(y_t[i][0])
print(accuracy_score(y_t, new_predicts))

0.14482758620689656


In [27]:
import random

random.randint(1,100)

17

In [2]:
i = 0
img = cv.imread(data_df.loc[i].fileName)
ratio=rand_scale = data_df.loc[i].ratio
sr=data_df.loc[i].sr
sc=data_df.loc[i].sc


old_size = img.shape[:2] # old_size is in (height, width) format

new_size = tuple([int(x*ratio) for x in old_size])

# new_size should be in (width, height) format

im = cv.resize(img, (new_size[1], new_size[0]))

top = int((old_size[0] - new_size[0])/2)
bottom = int((old_size[0] - new_size[0])/2)
left = int((old_size[1] - new_size[1])/2)
right = int((old_size[1] - new_size[1])/2)

if top+bottom+new_size[0] < old_size[0]:
    top += 1
if left+right+new_size[1] < old_size[1]:
    left += 1

color = [0, 0, 0]
img= cv.copyMakeBorder(im, top, bottom, left, right, cv.BORDER_CONSTANT,
    value=color)

rows,cols,_ = img.shape
M = np.float32([[1,0,sc],[0,1,sr]])
img = cv.warpAffine(img,M,(cols,rows))

for box in data_df.loc[i].boxx:
    cv.rectangle(img, (box[0],box[1]),(box[2],box[3]), (0,255,0), 10)

plt.figure(figsize=(30,20))
plt.imshow(img)
plt.show()




NameError: name 'cv' is not defined